### Dependencies

In [19]:
import json
import requests
import datetime
import pandas as pd
from IPython.display import HTML

This is the free query url provided by Copernicus

In [3]:
url = "https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json"

Provide a bounding box of the location of interest.
I use [this bounding box tool]("https://boundingbox.klokantech.com/").

In [6]:
# define your bounding box
bbox = "-93.329125,44.89015,-93.193859,45.05125"
# format expected: "min_lon,min_lat,max_lon,max_lat"

Provide a date range. By default we will look at the last seven days.

In [2]:
end_date = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
start_date = (datetime.datetime.utcnow() - datetime.timedelta(days=7)).strftime("%Y-%m-%dT%H:%M:%SZ")

Define your parameters... cloud cover may be all you care about for this query so let's just define that as a var.

In [4]:
# format possible: values from 0 to 10 (0 to 100% cloud cover)
cloud_cover = "[0,3]" # this will be 0 to 30% cover imagery only

The parameters all together

In [7]:
params = {
    "productType": "S2MSI2A",
    "cloudCover": cloud_cover,
    "startDate": start_date,
    "completionDate": end_date,
    "maxRecords": 10,
    "box": bbox,
}

Make the request itself:

In [9]:
response = requests.get(url, params=params)
data = response.json()

Print to console and save locally:

In [22]:
imagery = []
if "features" in data and data["features"]:
    print(f"Found {len(data['features'])} Sentinel-2 image(s) in the last week:")
    for feature in data["features"]:
        date_acquired, satellite, cloud_cover, thumbnail, download_url = [],[],[],[],[]
#         props = feature["properties"]
        props = feature.get("properties", {})
        imagery.append({
            "id": props.get("title"),
            "date_acquired": props.get("startDate"),
            "cloud_cover": props.get("cloudCover"),
            "thumbnail": props.get("thumbnail", "NaN"),
            "download_url": props.get("services", {}).get("download", {}).get("url", "NaN")
        })
        print(json.dumps(props, indent=4))
else:
    print("No Sentinel-2 images found for the specified bbox and time range.")
    
# make a df
df = pd.DataFrame(imagery)

# show thumbnails as images in the df
def render_thumb(url):
    return f'<img src="{url}" width="100"/>'

# make a preview of thumbnails
df["thumb_preview"] = df["thumbnail"].apply(render_thumb)

# save to csv locally
df.to_csv("latest_sentinel_images.csv", index=False)

Found 1 Sentinel-2 image(s) in the last week:
{
    "collection": "SENTINEL-2",
    "status": "ONLINE",
    "license": {
        "licenseId": "unlicensed",
        "hasToBeSigned": "never",
        "grantedCountries": null,
        "grantedOrganizationCountries": null,
        "grantedFlags": null,
        "viewService": "public",
        "signatureQuota": -1,
        "description": {
            "shortName": "No license"
        }
    },
    "parentIdentifier": null,
    "title": "S2C_MSIL2A_20250306T170211_N0511_R069_T15TVK_20250306T222015.SAFE",
    "description": "The Copernicus Sentinel-2 mission consists of two polar-orbiting satellites that are positioned in the same sun-synchronous orbit, with a phase difference of 180\u00b0. It aims to monitor changes in land surface conditions. The satellites have a wide swath width (290 km) and a high revisit time. Sentinel-2 is equipped with an optical instrument payload that samples 13 spectral bands: four bands at 10 m, six bands at 20 m 

In [26]:
# show preview of images
HTML(df.to_html(escape=False))

,id,date_acquired,cloud_cover,thumbnail,download_url,thumb_preview
0,S2C_MSIL2A_20250306T170211_N0511_R069_T15TVK_20250306T222015.SAFE,2025-03-06T17:02:11.025000Z,0.248171,https://catalogue.dataspace.copernicus.eu/get-object?path=/Sentinel-2/MSI/L2A/2025/03/06/S2C_MSIL2A_20250306T170211_N0511_R069_T15TVK_20250306T222015.SAFE/S2C_MSIL2A_20250306T170211_N0511_R069_T15TVK_20250306T222015-ql.jpg,https://catalogue.dataspace.copernicus.eu/download/3f5f1cc4-0440-4844-a436-48920025cc82,
